In [1]:
import cv2
import numpy as np
import math

# 2点を結ぶ方程式y=ax+bを求める
def get_equation(x1,y1,x2,y2):
    a = (y2-y1) / (x2-x1)
    b = y1 - (a*x1)
    return a,b

# 2方程式の交点を求める
def get_intersection(eq1,eq2):
    a = eq1[0]
    b = eq1[1]
    A = eq2[0]
    B = eq2[1]
    
    x = round((B-b) / (a-A))
    y = round(((a*B)-(A*b)) / (a-A))
    return x,y

# ベクトルの真下に対する角度を求める
def get_angle(vector):
    vector_down = np.array([0,1])
    vector = np.array(vector)

    angle = np.arccos(np.inner(vector,vector_down) / (np.linalg.norm(vector) * np.linalg.norm(vector_down)))
    return angle




img = cv2.imread('./images/DSC_1493.JPG')

# アスペクト比を固定して画像を変換
re_length = 500 # リサイズしたい長い辺のサイズ
h, w = img.shape[:2] # 縦横のサイズを取得
re_h = re_w = re_length/max(h,w) # 変換する倍率を計算
img = cv2.resize(img, dsize=None, fx=re_h , fy=re_w)

# 短辺の長さ
short_length = min(img.shape[:2])

# エッジ抽出
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,150,200,apertureSize = 3)

# ハフ変換（円検出）
# 最小直径は画像短辺の半分、最大直径は画像短辺
circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT,1,20,param1=120,param2=50, minRadius=round(short_length/4), maxRadius=round(short_length/2))
if circles is not None:
    circles = np.uint16(np.around(circles))
    circle = circles[0,:][0]
    
    cv2.circle(img, (circle[0],circle[1]),circle[2],(0,255,0),2)#外円描画
    cv2.circle(img, (circle[0],circle[1]),2,(0,0,255),1)#中心描画


    #円で切り抜き
    mask=np.zeros((img.shape[:2][0],img.shape[:2][1]),dtype=np.uint8)
    cv2.circle(mask, (circle[0],circle[1]), circle[2], color=255,thickness=-1)
    # img[mask==0] = [0, 0, 0]
    edges[mask==0] = 0
    cv2.imwrite("clipped.png", edges)

    # ハフ変換（直線検出）
    # 最小長さは円の半径の1/2
    # maxLineGapは円の半径の1/5
    circle_radius = circle[2]
    # print(round(circle_radius*0.75))
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/360, threshold=50, minLineLength=round(circle_radius*0.5), maxLineGap=round(circle_radius/5))

    for line in lines[:2]:
        x1, y1, x2, y2 = line[0]
        line_img = cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    

    x11, y11, x12, y12 = lines[0][0]
    x21, y21, x22, y22 = lines[1][0]

    eq1 = get_equation(x11, y11, x12, y12)
    eq2 = get_equation(x21, y21, x22, y22)

    intersection = get_intersection(eq1,eq2)#交点

    #交点描画
    cv2.circle(img, (intersection[0],intersection[1]),3,(255,0,0),1)
    needle_direction = (intersection[0] - circle[0], intersection[1] - circle[1])
    angle = get_angle(needle_direction)
    print(math.degrees(angle))

    cv2.imwrite("line.png", img)
    



# 表示
cv2.imshow("", img)
cv2.waitKey()
cv2.destroyAllWindows()

52.65065095535941


参考
https://www.higashisalary.com/entry/opencv-make-mask
